train.py

In [65]:
from __future__ import print_function
from scipy.misc import imsave, imresize
import numpy as np
import time
import os
import h5py

from keras.models import Sequential
from keras.layers import *
from keras.layers.advanced_activations import ELU
from keras import backend as K

from keras.models import Model

from keras.layers.convolutional import (
    Convolution2D,
    MaxPooling2D,
    AveragePooling2D
)

from keras.layers.normalization import BatchNormalization

from keras.utils import np_utils

import scipy
from scipy import ndimage
from scipy.ndimage import imread

from keras.datasets import mnist
from keras.models import model_from_json
import keras

from time import sleep

import random

import matplotlib.pyplot as plt
# %matplotlib inline

In [2]:
from helper import make_noise_img, make_box_img, deprocess_image, normalize, preprocess_image

In [4]:
# dimensions of the generated pictures 
img_width = 28
img_height = 28

In [5]:
# # input image dimensions
# img_rows, img_cols = 28, 28

# # the data, shuffled and split between train and test sets
# (X_train, y_train), (X_test, y_test) = mnist.load_data()

# X_train = X_train.reshape(X_train.shape[0], 1, img_rows, img_cols)
# X_test = X_test.reshape(X_test.shape[0], 1, img_rows, img_cols)
# X_train = X_train.astype('float32')
# X_test = X_test.astype('float32')
# X_train /= 255
# X_test /= 255
# print('X_train shape:', X_train.shape)
# print(X_train.shape[0], 'train samples')
# print(X_test.shape[0], 'test samples')

# # # convert class vectors to binary class matrices
# # Y_train = np_utils.to_categorical(y_train, nb_classes)
# # Y_test = np_utils.to_categorical(y_test, nb_classes)

X_train shape: (60000, 1, 28, 28)
60000 train samples
10000 test samples


In [6]:
# plt.imshow(X_test[0].squeeze())

In [7]:
#wave 1
from helper import boxify_top_left, boxify_top_right, boxify_bottom_left, boxify_bottom_right
from helper import lineify_top_left, lineify_bottom_right
from helper import circleify_top_left, circleify_bottom_right
from helper import triangulify_top_left, triangulify_bottom_right
#wave 2
from helper import boxify_center, lineify_center, circleify_center, triangulify_center, noiseify
from helper import boxify_center_hollow, lineify_center_horizontal, circleify_center_hollow, triangulify_center_hollow, noiseify_blur
#wave 3
from helper import boxify, lineify, circleify

load training data

In [86]:
# 0 = do not include in training data
# 1 = include in training data

def preprocess(training_data_indicies):
    x_data = []
    y_data = []

    num_of_pictures = 3 #5000*2

    blank = np.zeros([1,28,28])
    
    num_total_training_images = len(training_data_indicies)
    
    for i in range(num_of_pictures):
        
        # row 1
        if training_data_indicies[0%num_total_training_images] == 1:
            x_data.append(boxify_center(np.copy(blank)))
            y_data.append(0)
            print(0)

        if training_data_indicies[1%num_total_training_images] == 1:
            x_data.append(boxify_center_hollow(np.copy(blank)))
            y_data.append(1)
            print(1)
            
        if training_data_indicies[2%num_total_training_images] == 1:
            x_data.append(lineify_center(np.copy(blank)))
            y_data.append(2)
            print(2)
            
        if training_data_indicies[3%num_total_training_images] == 1:
            x_data.append(lineify_center_horizontal(np.copy(blank)))
            y_data.append(3)
            print(3)
            
        if training_data_indicies[4%num_total_training_images] == 1:
            x_data.append(circleify_center(np.copy(blank)))
            y_data.append(4)
            print(4)
                        
        if training_data_indicies[5%num_total_training_images] == 1:
            x_data.append(circleify_center_hollow(np.copy(blank)))
            y_data.append(5)
            
        if training_data_indicies[6%num_total_training_images] == 1:
            x_data.append(triangulify_center(np.copy(blank)))
            y_data.append(6)
            
        if training_data_indicies[7%num_total_training_images] == 1:
            x_data.append(triangulify_center_hollow(np.copy(blank)))
            y_data.append(7)
            
        # row 2
        if training_data_indicies[8%num_total_training_images] == 1:
            x_data.append(boxify_top_left(np.copy(blank)))
            y_data.append(8)

        if training_data_indicies[9%num_total_training_images] == 1:
            x_data.append(boxify_bottom_right(np.copy(blank)))
            y_data.append(9)
            
        if training_data_indicies[10%num_total_training_images] == 1:
            x_data.append(lineify_top_left(np.copy(blank)))
            y_data.append(10)

        if training_data_indicies[11%num_total_training_images] == 1:
            x_data.append(lineify_bottom_right(np.copy(blank)))
            y_data.append(11)
            
        if training_data_indicies[12%num_total_training_images] == 1:
            x_data.append(circleify_top_left(np.copy(blank)))
            y_data.append(12)

        if training_data_indicies[13%num_total_training_images] == 1:
            x_data.append(circleify_bottom_right(np.copy(blank)))
            y_data.append(13)
            
        if training_data_indicies[14%num_total_training_images] == 1:
            x_data.append(triangulify_top_left(np.copy(blank)))
            y_data.append(14)

        if training_data_indicies[15%num_total_training_images] == 1:
            x_data.append(triangulify_bottom_right(np.copy(blank)))
            y_data.append(15)
            
        # row 3
        if training_data_indicies[16%num_total_training_images] == 1:
            x_data.append(noiseify())
            y_data.append(16)
            
        if training_data_indicies[17%num_total_training_images] == 1:
            x_data.append(noiseify_blur())
            y_data.append(17)
        
    nb_classes = num_total_training_images

    X_train_new = np.array(x_data)
    y_train_new = np.array(y_data)
    Y_train_new = np_utils.to_categorical(y_train_new, nb_classes)

    import random
    s=list(range(X_train_new.shape[0]))
    random.shuffle(s) 
    X2 = X_train_new[s]+np.random.random(X_train_new.shape)*0.01
    Y2 = Y_train_new[s]
    
    return X2, Y2

In [87]:
test = np.array([1,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0])
X2, Y2 = preprocess(test)
print(X2.shape)
print(Y2.shape)

0
2
3
0
2
3
0
2
3
(9, 1, 28, 28)
(9, 18)


In [88]:
X2.shape
Y2.shape

(9, 18)

**MNIST model has been converted to functional API, need to load MNIST weights here**

In [90]:
batch_size = 4 #32
nb_epoch = 50 #5*2

# input image dimensions
img_rows, img_cols = 28, 28
WIDTH = 64*2
num_layers = 8

input = Input(batch_shape=(batch_size, 1, img_rows, img_cols))
# L0 = Flatten()(input)
# L1 = Dense(WIDTH,activation='tanh')(L0)
# L1 = BatchNormalization(mode=2)(L1)
# m = L1
# #L2 = Dense(WIDTH,activation='sigmoid')(L1)
# for i in range(num_layers):
#     m = BatchNormalization(mode=2)(m)
#     m = Dense(WIDTH/2,activation='tanh')(m)
    
# m = BatchNormalization(mode=2)(m)
# m = Dense(WIDTH,activation='tanh')(m)
# #L3 = merge([model,L1],mode='concat')
# L3 = Dense(WIDTH,activation='linear')(m)
# L3 = BatchNormalization(mode=2)(L3)
# final = Dense(nb_classes,activation='softmax')(L3)
# model = Model(input=input,output=[final], name='m2_snn')
# # firstlayer = Model(input=input,output=L1)
# # lastlayer = Model(input=input,output=L3)

m = Flatten()(input)
m = Dense(WIDTH, activation='tanh')(m)
# m = Dropout(0.2)(m)
m = Dense(WIDTH, activation='tanh')(m)
m = Dense(nb_classes, activation='softmax')(m)

model = Model(input=input, output=[m])

model.compile(loss='categorical_crossentropy',
              optimizer='sgd',
              metrics=['accuracy'])

In [91]:
# modelone.__dict__

In [92]:
print(model.summary())

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_9 (InputLayer)             (4, 1, 28, 28)        0                                            
____________________________________________________________________________________________________
flatten_9 (Flatten)              (4, 784)              0           input_9[0][0]                    
____________________________________________________________________________________________________
dense_25 (Dense)                 (4, 128)              100480      flatten_9[0][0]                  
____________________________________________________________________________________________________
dense_26 (Dense)                 (4, 128)              16512       dense_25[0][0]                   
___________________________________________________________________________________________

run model

In [93]:
from keras.callbacks import ModelCheckpoint
# checkpointer = ModelCheckpoint(filepath="/Users/fredhohman/Desktop/pnnl/ubuntu-vm/Desktop/Github/pnnl-summer-research-notes/output/mnist_box_shape_snn" + str(loop_num) + ".hdf5", verbose=1, save_best_only=True)
# earlystop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=0, verbose=0, mode='auto')
model.fit(X2,Y2,batch_size=batch_size,nb_epoch=nb_epoch,validation_split=0.2,shuffle=True, verbose=2)
sleep(0.1)

Train on 7 samples, validate on 2 samples
Epoch 1/100
0s - loss: 3.0489 - acc: 0.0000e+00 - val_loss: 2.4333 - val_acc: 0.5000
Epoch 2/100
0s - loss: 2.2050 - acc: 0.5714 - val_loss: 2.1721 - val_acc: 0.5000
Epoch 3/100
0s - loss: 1.6321 - acc: 0.7143 - val_loss: 1.9125 - val_acc: 1.0000
Epoch 4/100
0s - loss: 1.2912 - acc: 1.0000 - val_loss: 1.6799 - val_acc: 1.0000
Epoch 5/100
0s - loss: 1.1205 - acc: 1.0000 - val_loss: 1.4650 - val_acc: 1.0000
Epoch 6/100
0s - loss: 0.9340 - acc: 1.0000 - val_loss: 1.2776 - val_acc: 1.0000
Epoch 7/100
0s - loss: 0.8210 - acc: 1.0000 - val_loss: 1.1278 - val_acc: 1.0000
Epoch 8/100
0s - loss: 0.7341 - acc: 1.0000 - val_loss: 1.0072 - val_acc: 1.0000
Epoch 9/100
0s - loss: 0.6261 - acc: 1.0000 - val_loss: 0.8792 - val_acc: 1.0000
Epoch 10/100
0s - loss: 0.5566 - acc: 1.0000 - val_loss: 0.7776 - val_acc: 1.0000
Epoch 11/100
0s - loss: 0.4874 - acc: 1.0000 - val_loss: 0.6842 - val_acc: 1.0000
Epoch 12/100
0s - loss: 0.4323 - acc: 1.0000 - val_loss: 0.60

In [42]:
# json_string = extractor.to_json()
# outfile = open("output/mnist_box_shape_classifier_only_no_weights_ex.json",'w')
# outfile.write(json_string)
# outfile.close()
# extractor.save_weights('output/mnist_box_shape_classifier_only_no_weights_ex.h5', overwrite=True)

In [43]:
json_string = model.to_json()
outfile = open("output/mnist_box_shape_snn.json",'w')
outfile.write(json_string)
outfile.close()
model.save_weights('output/mnist_box_shape_snn.h5', overwrite=True)

In [81]:
index = 0
blankp = np.zeros([1,28,28])
# print(np.around(model.predict(boxify_center(np.copy(blankp))[None,])))
# print(np.around(model.predict(lineify_center(np.copy(blankp))[None,])))
# print(np.around(model.predict(circleify_center(np.copy(blankp))[None,])))
# print(np.around(model.predict(triangulify_center(np.copy(blankp))[None,])))
# print(np.around(model.predict(noiseify()[None,])))


temp = np.array([boxify_center(np.copy(blankp))])
model.predict(temp)

array([[  8.77370477e-01,   1.66038591e-02,   7.15982681e-03,
          5.61637850e-03,   6.07573204e-02,   2.43873522e-03,
          1.56016462e-02,   4.98978468e-03,   4.00792691e-04,
          8.81121901e-04,   4.13888833e-04,   5.04756754e-04,
          5.52214740e-04,   1.54533796e-03,   2.70666176e-04,
          8.26605421e-04,   2.07248679e-03,   1.99408084e-03]], dtype=float32)